In [40]:
#Install Packages
!pip install faiss-cpu
!pip install sentence-transformers

  Using cached numpy-2.1.3-cp311-cp311-win_amd64.whl.metadata (60 kB)
Using cached numpy-2.1.3-cp311-cp311-win_amd64.whl (12.9 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5


  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.0 requires FuzzyTM>=0.4.0, which is not installed.
tables 3.8.0 requires blosc2~=2.0.0, which is not installed.
tables 3.8.0 requires cython>=0.29.21, which is not installed.
langchain 0.3.8 requires numpy<2,>=1.22.4; python_version < "3.12", but you have numpy 2.1.3 which is incompatible.
langchain-community 0.3.8 requires numpy<2,>=1.22.4; python_version < "3.12", but you have numpy 2.1.3 which is incompatible.
numba 0.57.1 requires numpy<1.25,>=1.21, but you have numpy 2.1.3 which is incompatible.
scipy 1.11.1 requires numpy<1.28.0,>=1.21.6, but you have numpy 2.1.3 which is incompatible.
tensorflow-intel 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 2.1.3 which is incompatible.
unstructured 0.16.8 requires numpy<2, but you have numpy 2.1.3 which is incomp

  Using cached numpy-1.26.4-cp311-cp311-win_amd64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-win_amd64.whl (15.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.3
    Uninstalling numpy-2.1.3:
      Successfully uninstalled numpy-2.1.3


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.0 requires FuzzyTM>=0.4.0, which is not installed.
tables 3.8.0 requires blosc2~=2.0.0, which is not installed.
tables 3.8.0 requires cython>=0.29.21, which is not installed.
numba 0.57.1 requires numpy<1.25,>=1.21, but you have numpy 1.26.4 which is incompatible.
tensorflow-intel 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 1.26.4 which is incompatible.


In [41]:
#import necessary libaries
import pandas as pd

In [42]:
df=pd.read_csv("sample_text.csv")
df

,text,category
0,Meditation and yoga can improve mental health,Health
1,"Fruits, whole grains and vegetables helps cont...",Health
2,These are the latest fashion trends for this week,Fashion
3,Vibrant color jeans for male are becoming a trend,Fashion
4,The concert starts at 7 PM tonight,Event
5,Navaratri dandiya program at Expo center in Mu...,Event
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity i...,Travel


In [43]:
df.shape

(8, 2)

#### Step 1: Create source Embeddings for Text Column

In [44]:
from sentence_transformers import SentenceTransformer

In [45]:
encoder=SentenceTransformer("all-mpnet-base-v2")


In [46]:
vectors=encoder.encode(df.text)

In [47]:
vectors

array([[-0.00247386,  0.03626738, -0.05290457, ..., -0.09152357,
        -0.03970006, -0.04330486],
       [-0.03357265,  0.0098052 , -0.03250131, ..., -0.05165471,
         0.02245886, -0.0315618 ],
       [-0.01865333, -0.04051305, -0.01235392, ...,  0.00610591,
        -0.07179645,  0.02773852],
       ...,
       [-0.00066453,  0.04252131, -0.05645506, ...,  0.0131547 ,
        -0.03183563, -0.04357663],
       [-0.03317152,  0.03252462, -0.02484841, ...,  0.01174419,
         0.0574712 ,  0.00571026],
       [-0.00166399,  0.00413823, -0.04597077, ...,  0.02008529,
         0.05656242, -0.00161591]], dtype=float32)

In [48]:
vectors.shape

(8, 768)

In [49]:
vectors[0]

array([-2.47385865e-03,  3.62673812e-02, -5.29045686e-02, -2.17348412e-02,
        1.41724274e-02, -1.01591100e-03,  2.23636422e-02,  3.65195721e-02,
        3.80900763e-02,  5.22010261e-03, -3.00865527e-02,  2.05449737e-03,
        1.08502787e-02, -8.49088803e-02, -5.57267945e-03, -6.06598891e-02,
        2.32001580e-03, -1.51408454e-02,  1.20671494e-02,  1.98905319e-02,
       -1.27935484e-02, -2.72811223e-02, -3.31878923e-02,  3.40651278e-03,
        8.41173343e-03, -1.94854308e-02,  4.88132238e-02,  2.24118605e-02,
        4.53466661e-02, -5.90892062e-02, -9.13182423e-02, -1.22329080e-03,
       -4.05045450e-02,  8.67181830e-03,  1.26565294e-06, -1.92451440e-02,
       -3.20658907e-02, -4.08172496e-02,  8.79176855e-02, -1.52631504e-02,
        1.09909270e-02, -4.47977036e-02, -4.74819634e-03,  3.52632739e-02,
        1.31070642e-02,  1.09943300e-02, -3.45756859e-03,  9.12349001e-02,
        5.68808839e-02,  4.82893363e-02, -4.51440038e-03,  3.58451083e-02,
       -2.44209599e-02, -

In [50]:
vectors[0].shape

(768,)

In [51]:
dim=vectors.shape[1]
dim 

768

### Step 2 : Build a FAISS Index for vectors

In [52]:
import faiss

index=faiss.IndexFlatL2(dim)

### Step 3 : Normalize the source vectors ( using L2 distance to measure similarity) and add to the index

In [53]:
index.add(vectors)

In [54]:
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x000001BC9E250990> >

### Step 4 : Encode search text using same encorder and normalize the output vector

In [55]:
#search_query="I want to buy a polo t-shirt"
search_query = "looking for places to visit during the holidays"
# search_query = "An apple a day keeps the doctor away

vec=encoder.encode(search_query)
vec.shape

(768,)

In [56]:
import numpy as np
svec=np.array(vec).reshape(1,-1)
svec.shape

(1, 768)

In [57]:
#faiss.normalize_L2(svec)

### Step 5: Search for similar vector in the FAISS index created

In [58]:
distances,I = index.search(svec,k=2)
distances

array([[0.92737234, 1.1601744 ]], dtype=float32)

In [59]:
I

array([[6, 7]], dtype=int64)

In [60]:
df.loc[I[0]]

,text,category
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity i...,Travel


In [62]:
search_query

'looking for places to visit during the holidays'

**The two results from the dataframe are similar to a search_query**